In [ ]:
%pip install ultralytics

In [ ]:
%pip install mediapipe

## 데이터 학습

In [ ]:
from ultralytics import YOLO

# 데이터 경로 및 설정
data_yaml_path = r"C:\Users\user\Documents\AI_project_clone\dataset\data.yaml"  # YAML 파일 경로 (수정 필요)
model_checkpoint = 'yolov8n.pt'  # YOLOv8 Nano 모델 가중치 (기본값)

# YOLO 모델 초기화
model = YOLO(model_checkpoint)


# 모델 학습
model.train(
    data=data_yaml_path,
    epochs=50,
    imgsz=640,
    batch=16,
    lr0=0.005,  # 초기 학습률 설정
    # lrf=0.2, # 최종학습율 비율
    name='helmet_detection_model',
    project='helmet_detection'
    
)


print("학습완료")


## 최종코드

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO
import os

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\사용자인증.jpeg"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image, (300, 300))

# 다운로드된 Streamlit 파일 경로
streamlit_file_path = r"C:\Users\user\Downloads\streamlit_download.txt"

# 웹캠 열기
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detected_time = None
qr_displayed_time = None
authentication_complete_time = None
authentication_completed = False  # 인증 완료 상태 플래그

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전ㅈㅈ
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    # YOLO로 헬멧 탐지
    results = model.predict(source=frame, save=False, show=False)
    helmet_detected = False
    no_helmet_detected = True
    only_helmet_detected = False
    
    # YOLO 결과 처리
    if not authentication_completed:
        if results[0].boxes:  # YOLO가 객체를 감지한 경우
            no_helmet_detected = False  # 객체가 감지되면 초기값을 업데이트

            for box in results[0].boxes:
                cls = int(box.cls)
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                if cls == 1 and conf > 0.4:  # With-helmet
                    helmet_detected = True
                    color = (0, 255, 0)  # 초록색
                    label = "With helmet"
                elif cls == 0 and conf > 0.5:  # No-helmet
                    no_helmet_detected = True
                    color = (0, 0, 255)  # 빨간색
                    label = "No helmet"
                elif cls == 2 and conf > 0.5:  # With cap
                    color = (0, 255, 255)  # 노란색
                    label = "With cap"
                elif cls == 3 and conf > 0.3:  # Only helmet
                    only_helmet_detected = True
                    color = (255, 0, 0)  # 파란색
                    label = "Only helmet"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(
                    frame,
                    f"{label} ({conf:.2f})",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    color,
                    2,
                )

        if landmark_points:
            landmarks = landmark_points[0].landmark
            left_eye = [landmarks[145], landmarks[159]]
            for landmark in left_eye:
                x = int(landmark.x * frame_w)
                y = int(landmark.y * frame_h)
                cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

            # 왼쪽 눈 깜빡임 감지
            if abs(left_eye[0].y - left_eye[1].y) < 0.005:
                eye_closed = True

            # 눈 깜빡임 로직
            if helmet_detected and eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
                blink_count += 1
                last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 메시지 출력 (우선 순위 설정)
    if not authentication_completed:  # 인증 완료되지 않은 경우에만 실행
        if no_helmet_detected:
            cv2.putText(frame, "Please wear a helmet", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Please wear a full helmet!", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)
        elif helmet_detected:
            if blink_count < 2:  # 윙크가 아직 두 번 감지되지 않은 경우
                cv2.putText(frame, "Blink twice to start helmet detection", 
                            (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
            elif blink_count >= 2:  # 윙크가 두 번 감지된 경우
                if qr_displayed_time is None:  # QR 창이 표시된 적 없을 때만
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시
                    blink_count = 0  # 깜빡임 초기화
        elif qr_displayed_time:  # QR 창이 닫혔는지 확인
            if cv2.getWindowProperty("Scan QR Code", cv2.WND_PROP_VISIBLE) < 1:
                cv2.imshow("Scan QR Code", qr_image)  # QR 코드 다시 표시


    # QR 코드 인증 완료 확인
    if qr_displayed_time and not authentication_completed:
        if os.path.exists(streamlit_file_path):
            try:
                with open(streamlit_file_path, 'r') as file:
                    content = file.read().strip()
                    if content == "0":
                        authentication_completed = True  # 인증 완료 상태 설정
                        qr_displayed_time = None
                        cv2.destroyWindow("Scan QR Code")  # QR 창 닫기

            except Exception as e:
                print(f"Error reading file: {e}")

    # 인증 완료 상태 메시지
    if authentication_completed:
        cv2.putText(frame, "Authentication Complete!", (50, 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)

        results = model.predict(source=frame, conf=0.4, save=False, show=False)
        helmet_worn = False
        only_helmet_detected = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1 and conf > 0.4:  # With-helmet
                helmet_worn = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 2 and conf > 0.4:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 0 and conf > 0.6:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
            elif cls == 3 and conf > 0.3:  # Only helmet
                only_helmet_detected = True
                color = (255, 0, 0)  # 파란색
                label = "Only helmet"

            # 바운딩 박스와 레이블 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2,
            )

        # 헬멧 착용 상태에 따른 메시지
        if helmet_worn:
            cv2.putText(frame, "You're wearing a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Only Helmet detected.", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)
        else:
            cv2.putText(frame, "Please wear a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

## 진짜 최종

In [ ]:
import cv2
import time
import mediapipe as mp
from ultralytics import YOLO
import os

# YOLO 모델 로드
model = YOLO(r"C:\Users\user\Documents\AI_project_clone\helmet_detection\helmet_detection_model\weights\best.pt")

# Mediapipe Face Mesh 초기화
face_mesh = mp.solutions.face_mesh.FaceMesh(refine_landmarks=True)

# QR 코드 이미지 경로
qr_image_path = r"C:\Users\user\Documents\AI_project_clone\사용자인증.jpeg"
qr_image = cv2.imread(qr_image_path)  # QR 코드 이미지 로드
qr_image = cv2.resize(qr_image, (300, 300))

# 다운로드된 Streamlit 파일 경로
streamlit_file_path = r"C:\Users\user\Downloads\streamlit_download.txt"

# 웹캠 열기
cap = cv2.VideoCapture(0)
FRAME_WIDTH = 1280  # 너비
FRAME_HEIGHT = 720  # 높이
cap.set(cv2.CAP_PROP_FRAME_WIDTH, FRAME_WIDTH)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, FRAME_HEIGHT)

if not cap.isOpened():
    print("Error: Could not open webcam.")
    exit()

# 눈 깜빡임 감지 상태 변수 초기화
blink_count = 0  # 깜빡임 횟수
last_blink_time = None  # 마지막 깜빡임 시간
BLINK_DELAY = 2  # 눈 깜빡임 간격 제한 (초)

# 헬멧 감지 상태 변수 초기화
helmet_detected_time = None
qr_displayed_time = None
authentication_complete_time = None
authentication_completed = False  # 인증 완료 상태 플래그

while True:
    ret, frame = cap.read()
    if not ret:
        print("Error: Failed to capture image.")
        break

    frame = cv2.flip(frame, 1)  # 좌우 반전
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe로 얼굴 인식 및 눈 깜빡임 감지
    output = face_mesh.process(rgb_frame)
    landmark_points = output.multi_face_landmarks
    frame_h, frame_w, _ = frame.shape

    eye_closed = False

    # YOLO로 헬멧 탐지
    results = model.predict(source=frame, save=False, show=False)
    helmet_detected = False
    no_helmet_detected = True
    only_helmet_detected = False
    cap_detected = False

    # YOLO 결과 처리
    if not authentication_completed:
        if results[0].boxes:  # YOLO가 객체를 감지한 경우
            no_helmet_detected = False  # 객체가 감지되면 초기값을 업데이트

            for box in results[0].boxes:
                cls = int(box.cls)
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])

                if cls == 1 and conf > 0.4:  # With-helmet
                    helmet_detected = True
                    color = (0, 255, 0)  # 초록색
                    label = "With helmet"
                elif cls == 0 and conf > 0.5:  # No-helmet
                    no_helmet_detected = True
                    color = (0, 0, 255)  # 빨간색
                    label = "No helmet"
                elif cls == 2 and conf > 0.5:  # With cap
                    cap_detected =True
                    color = (0, 255, 255)  # 노란색
                    label = "With cap"
                elif cls == 3 and conf > 0.3:  # Only helmet
                    only_helmet_detected = True
                    color = (255, 0, 0)  # 파란색
                    label = "Only helmet"

                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(
                    frame,
                    f"{label} ({conf:.2f})",
                    (x1, y1 - 10),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.6,
                    color,
                    2,
                )
        if landmark_points:
            landmarks = landmark_points[0].landmark
            left_eye = [landmarks[145], landmarks[159]]
            for landmark in left_eye:
                x = int(landmark.x * frame_w)
                y = int(landmark.y * frame_h)
                cv2.circle(frame, (x, y), 3, (0, 255, 255))  # 왼쪽 눈 표시

            # 왼쪽 눈 깜빡임 감지
            if abs(left_eye[0].y - left_eye[1].y) < 0.005:
                eye_closed = True

            # 눈 깜빡임 로직
            if helmet_detected and eye_closed and (last_blink_time is None or time.time() - last_blink_time > BLINK_DELAY):
                blink_count += 1
                last_blink_time = time.time()  # 마지막 깜빡임 시간 기록

    # 메시지 출력 (우선 순위 설정)
    if not authentication_completed:  # 인증 완료되지 않은 경우에만 실행
        if no_helmet_detected:
            cv2.putText(frame, "Please wear a helmet", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Please wear a full helmet!", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (255, 0, 0), 3)
        elif cap_detected:
            cv2.putText(frame, "Please wear a helmet", (50, 40), cv2.FONT_HERSHEY_SIMPLEX, 1.2, (0, 0, 255), 3)
        elif helmet_detected:
            if blink_count < 2:  # 윙크가 아직 두 번 감지되지 않은 경우
                cv2.putText(frame, "Please Blink twice",
                            (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 255), 2)
            elif blink_count >= 2:  # 윙크가 두 번 감지된 경우
                if qr_displayed_time is None:  # QR 창이 표시된 적 없을 때만
                    qr_displayed_time = time.time()  # QR 코드 표시 시작 시간
                    cv2.imshow("Scan QR Code", qr_image)  # QR 코드 표시
                    blink_count = 0  # 깜빡임 초기화
        elif qr_displayed_time:  # QR 창이 닫혔는지 확인
            if cv2.getWindowProperty("Scan QR Code", cv2.WND_PROP_VISIBLE) < 1:
                cv2.imshow("Scan QR Code", qr_image)  # QR 코드 다시 표시


    # QR 코드 인증 완료 확인
    if qr_displayed_time and not authentication_completed:
        if os.path.exists(streamlit_file_path):
            try:
                with open(streamlit_file_path, 'r') as file:
                    content = file.read().strip()
                    if content == "0":
                        authentication_completed = True  # 인증 완료 상태 설정
                        qr_displayed_time = None
                        cv2.destroyWindow("Scan QR Code")  # QR 창 닫기

            except Exception as e:
                print(f"Error reading file: {e}")

    # 인증 완료 상태 메시지
    if authentication_completed:
        cv2.putText(frame, "Authentication Complete!", (50, 20), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)

        results = model.predict(source=frame, conf=0.4, save=False, show=False)
        helmet_worn = False
        only_helmet_detected = False

        for box in results[0].boxes:
            cls = int(box.cls)
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = float(box.conf[0])

            if cls == 1 and conf > 0.4:  # With-helmet
                helmet_worn = True
                color = (0, 255, 0)  # 초록색
                label = "With helmet"
            elif cls == 2 and conf > 0.4:  # With cap
                color = (0, 255, 255)  # 노란색
                label = "With cap"
            elif cls == 0 and conf > 0.6:  # No-helmet
                color = (0, 0, 255)  # 빨간색
                label = "No helmet"
            elif cls == 3 and conf > 0.3:  # Only helmet
                only_helmet_detected = True
                color = (255, 0, 0)  # 파란색
                label = "Only helmet"

            # 바운딩 박스와 레이블 표시
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(
                frame,
                f"{label} ({conf:.2f})",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.6,
                color,
                2
            )

        # 헬멧 착용 상태에 따른 메시지
        if helmet_worn:
            cv2.putText(frame, "You're wearing a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 255, 0), 3)
        elif only_helmet_detected:
            cv2.putText(frame, "Only Helmet detected.", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 0, 0), 3)
        else:
            cv2.putText(frame, "Please wear a helmet!", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1.0, (0, 0, 255), 3)

    # 결과 화면 표시
    cv2.imshow("Helmet Detection", frame)

    # 'q' 키로 종료
    if cv2.waitKey(1) == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()